In [1]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
import numpy as np
from dataset_load import *

Using TensorFlow backend.


In [2]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

In [3]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [4]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
cv = CountVectorizer(min_df = 100, token_pattern=token, lowercase=True, binary=True)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [5]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [6]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [7]:
def layer_split(x):
    return tf.split(x,num_or_size_splits=human_terms_len,axis=1)

def layer_concat(x):
    return tf.concat(x, axis=1)

In [8]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

# split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)
split = Lambda(layer_split)(ht_input_layer)


# get the document prediction
label_layer = base_model(combined_input_layer)

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
#     dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones')(Multiply()([split[i], label_layer])))
    dense_layer.append(Lambda(lambda x: tf.nn.relu(x))(Multiply()([split[i], label_layer])))

# concat all the result   
# concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)
concat = Lambda(layer_concat, name='concatenate')(dense_layer)


# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3686)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3687      
Total params: 3,687
Trainable params: 3,687
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3686)         0                                            
_________________________________________

In [9]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [10]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [11]:
y_train_agreement.shape

(25000, 83)

In [12]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=1)

Train on 16667 samples, validate on 8333 samples
Epoch 1/1
16667/16667 [==============================] - 39s 2ms/step - loss: 0.4189 - acc: 0.6683 - val_loss: 0.3827 - val_acc: 0.7607


In [13]:
combined_model_history = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], 
                                            validation_data=([X_train[16667:], y_train_agreement[16667:]], y_train_original[16667:]),
                                            batch_size=1, epochs=1)

Train on 16667 samples, validate on 8333 samples
Epoch 1/1
16667/16667 [==============================] - 62s 4ms/step - loss: 0.7822 - acc: 0.2812 - val_loss: 0.7371 - val_acc: 0.3034


In [14]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

25000/25000 [==============================] - 2s 72us/step


[0.15664623194694519, 0.7808]

In [15]:
predict = combined_model.predict([X_test, y_test_agreement])

In [16]:
predict = predict.flatten()
predict.shape

(25000,)

In [17]:
predict_class = (predict > 0.5).astype('int32')
predict_class

array([1, 1, 0, ..., 0, 0, 0])

In [18]:
np.sum(y_test_original == predict_class)/len(predict_class)

0.7808

In [19]:
np.sum(y_test_original==0)

12500

In [20]:
score = combined_model.evaluate([X_train, y_train_agreement], y_train_original)
score

25000/25000 [==============================] - 2s 68us/step


[0.7043012392807007, 0.32064]

In [21]:
# independent weight here
# independent_weight_base_model_history
# independent_weight_combined_model_history

In [22]:
# var name
# shared_weight_base_model_history
# shared_weight_combined_model_history

In [23]:
net_weight = base_model.get_weights()

In [24]:
len(net_weight)

2

In [25]:
weight = net_weight[0].flatten()

In [26]:
net_weight[1].shape

(1,)

In [27]:
words = cv.get_feature_names()

In [28]:
indices = np.argsort(weight)

In [29]:
for i in indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

worst 	 -1.209
4/10 	 -1.202
waste 	 -1.033
2/10 	 -1.012
3/10 	 -0.989
dull 	 -0.856
awful 	 -0.814
boring 	 -0.804
terrible 	 -0.771
disappointment 	 -0.763


In [30]:
pos_indices = indices[::-1]
for i in pos_indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

7/10 	 1.430
8/10 	 1.260
excellent 	 0.921
9/10 	 0.887
rare 	 0.858
refreshing 	 0.802
superb 	 0.789
10/10 	 0.786
wonderfully 	 0.754
flawless 	 0.754


In [31]:
combined_net_weight = combined_model.get_weights()

In [32]:
len(combined_net_weight)

4

In [33]:
ht_weight = []
for i in range(len(combined_net_weight)):
    if i>1 and i < 85:
        print(combined_net_weight[i].flatten())
        ht_weight.append(combined_net_weight[i].flatten())
    else:
        print(i, combined_net_weight[i].shape)
ht_weight = np.asarray(ht_weight)

0 (3686, 1)
1 (1,)
[-0.73451054 -0.6284068  -0.9505546  -1.1358702   0.46423852  0.04981197
  1.1171229   1.1807038   0.7242865   1.1483732   1.3966488  -1.1808897
 -1.0903455  -1.7783191  -2.7056255  -0.9206071   1.4511423   0.90570945
  1.6481128  -0.7885182  -1.8449062   0.35521823 -1.196198   -1.1972173
 -1.1461791  -0.98793656 -0.62261826 -1.2124568   0.8334861   1.2041789
  1.6602336  -1.1154387   0.9297403   0.7136163   1.2330765  -0.5037291
  1.7488377  -1.6823839   0.7760265   0.76901215  1.9361054  -1.2051767
  0.7253017  -1.0621685  -0.8528969  -1.116272   -0.7365551  -0.48256397
  1.1440527  -0.84071815 -1.1812139  -0.33688742  0.8324818  -0.6040537
 -0.99793476  1.2820325   0.5841155  -1.300786   -1.9235228  -0.9018619
 -0.92655     1.178293    0.69555706 -0.7385146   0.85668033 -1.3058534
 -0.8616819   1.0619226  -1.681982    0.8109647   1.1585295   0.61250424
 -0.5956988  -1.5814114  -1.2903956  -0.8107011  -2.1460977  -0.9080422
 -1.01583     1.4303803   0.5009269  -1.6

In [34]:
sigmoid_weight = combined_net_weight[85].flatten()

IndexError: list index out of range

In [ ]:
ht_weight = ht_weight.flatten()

In [ ]:
human_term_indices = np.argsort(ht_weight)

print('human terms \t dense_relu weight \t sigmoid weight \t\t pos/neg?')
for i in human_term_indices:
    print('%s \t %.3f \t %.3f \t\t %.1f' %(word_list[i], ht_weight[i], sigmoid_weight[i], connotation[word_list[i]]))

In [ ]:
len(connotation)

In [ ]:
# Print report on the word transparency


index = [9, 19]
def report():
    for i in index:
        print()
        bm = base_model.predict(X_test[i])
        
        cm = combined_model.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
        print(X_test_original[i], '\n\n actual label : ', y_test_original[i], '\n predict from base model : ', bm.flatten(), '\n predict label : ', cm.flatten())
    
        for i,output in enumerate(concat_output.flatten()):
            if output != 0:
                print(word_list[i], output)

In [ ]:
report()

In [ ]:
report()

In [ ]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)

In [ ]:
score

In [ ]:
# 

### Let's see the weight when the trainable is false

In [ ]:
# def build_combined_model():
#     # build the combined model
#     # Combined model
#     human_terms_len = len(word_list)

#     base_model = build_base_model(X_train.shape[1])

#     combined_input_layer = Input(shape=(X_train.shape[1],))

#     # build the hard coded weight for human terms
#     ht_input_layer = Input(shape=(human_terms_len,))

#     split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

#     # get the document prediction
#     label_layer = base_model(combined_input_layer)

#     # stack the multiply layer
#     dense_layer = []
#     for i in range(human_terms_len):
#         dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones', trainable=True)(Multiply()([split[i], label_layer])))

#     # concat all the result   
#     concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

#     # pass it to sigmoid layer
#     output_layer = Dense(1, activation='sigmoid')(concat)

#     combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
#     combined_model.summary()
    
#     return base_model, combined_model

In [ ]:
# base_model, combined_model = build_combined_model()

In [ ]:
# base_model.compile(loss='mse',
#                   optimizer='Adagrad',
#                   metrics=['acc'])

# combined_model.compile(loss='mse',
#                       optimizer='Adagrad',
#                       metrics=['acc'])

# base_model_history_train_false = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
#                                     validation_data=(X_train[16667:], y_train_tanh[16667:]),
#                                     batch_size=1, epochs=50)

# combined_model_history_train_false = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], batch_size=1, epochs=50)

In [ ]:
# base_model_history_trainfalse = base_model_history
# combined_model_history_trainfalse = combined_model_history

In [ ]:
# net_weights = combined_model.get_weights()

In [ ]:
# Evaluate model

### Generate color weighted

In [ ]:
import re

class ColoredWeightedDoc(object):
    def __init__(self, doc, human_terms, ht_weights, token_pattern=r"(?u)\b\w\w+\b", binary = False):
        self.doc = doc
        self.human_terms = human_terms
        self.ht_weights = ht_weights
        self.binary = binary
        self.tokenizer = re.compile(token_pattern)
#         self.abs_ranges = np.linspace(0, max([abs(coefs.min()), abs(coefs.max())]), 8)
    def _repr_html_(self):
        html_rep = ""
        tokens = self.doc.split(" ") 
        if self.binary:
            seen_tokens = set()       
        for token in tokens:
            vocab_tokens = self.tokenizer.findall(token.lower())
            if len(vocab_tokens) > 0:
                vocab_token = vocab_tokens[0]
                try:
                    vocab_index = self.human_terms.index(vocab_token)
                    
                    if not self.binary or vocab_index not in seen_tokens:
                        
                        if self.ht_weights[vocab_index] == 0: # human-terms which has been washed out (opposing)
                            html_rep = html_rep + "<font size = 2, color=lightgreen> " + token + " </font>"
                        
                        elif self.ht_weights[vocab_index] != 0: # human-terms transparency
                            html_rep = html_rep + "<font size = 3, color=blue> " + token + " </font>"
                        
                        else: # neutral word
                            html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                        
                        if self.binary:    
                            seen_tokens.add(vocab_index)
                    
                    else: # if binary and this is a token we have seen before
                        html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                except: # this token does not exist in the vocabulary
                    html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
            else:
                html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
        return html_rep

In [ ]:
# review = list(['I was in Chicago last week. And I wanted to see a movie so bad. I am surprised that the movie itself is just amazing.  The plot was kinda weak, but it was great.  Christopher Nolan is just brilliant. Never fails to amaze. Even though the weather was terrible, but I enjoyed it. No regret! Overall, 10/10. Recommended.'])

review = list(['I went to the movie. The movie is terrible. Acting was not too bad. The weather is okay. Avoid it.'])

In [ ]:
# lr1 = LogisticRegression(C=0.01, random_state=42, penalty='l1')
# lr1.fit(X_train, y_train_original)

review_cv = cv.transform(review)
# words = cv.get_feature_names()
# weights = lr1.coef_.flatten()



In [ ]:
from IPython import display

bm = base_model.predict(review_cv)
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

In [ ]:
from IPython import display
idx = 1001
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

In [ ]:
np.where(np.sum(y_test_agreement, axis=1)!=0)[0][1000:1010]

In [ ]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

In [ ]:
concat_all = concat_after_relu.predict([X_test, y_test_agreement])

In [ ]:
concat_all.shape

In [ ]:
indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

In [ ]:
not_zero_predict = combined_model.evaluate([X_test[indices], y_test_agreement[indices]], y_test_original[indices])

In [ ]:
not_zero_predict

In [ ]:
(X_test.shape[0] - len(indices))/X_test.shape[0]

In [ ]:
combined_model.metrics_names

In [ ]:
combined_model.save_weights('imdb_human_term_weight.hdf5')

# Get cases

In [ ]:
from IPython import display
idx = 7079
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

In [ ]:
from IPython import display
idx = 11050
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

In [ ]:
from IPython import display
idx = 21625
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output, connotation[word_list[i]])


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

In [ ]:
from IPython import display
idx = 2494
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

In [ ]:
y_test_agreement[14689]

In [ ]:
positive = np.sum(y_test_agreement==1, axis=1)

In [ ]:
negative = np.sum(y_test_agreement==-1, axis=1)

In [ ]:
np.argmax(positive)

In [ ]:
np.max(negative)

In [ ]:
X_train_original[np.argmax(negative)]

In [ ]:
np.argmax(negative)

In [ ]:
np.argmax(positive)

In [ ]:
positive = positive.reshape(25000, 1)
negative = negative.reshape(25000, 1)

stack = np.hstack([positive, negative])
stack.shape

In [ ]:
new = np.absolute(stack[:,0] - stack[:,1])

In [ ]:
ind = []
for i in range(new.shape[0]):
    if new[i] <= 1 and (stack[i,0] >6  or stack[i,1] >6):
        ind.append(i)

In [ ]:
ind

In [ ]:
stack_balance = []
for i in ind:
    stack_balance.append(stack[i])

In [ ]:
stack_balance

In [ ]:
np.argmax(stack_balance)

In [ ]:
stack_balance[510]

In [ ]:
np.argmax(np.sum(stack, axis=1))